# 🏥 Visual AI in Healthcare with FiftyOne - 3D CT Segmentation with NVIDIA VISTA and FiftyOne
**Empowering medical imaging workflows with open-source tools and modern AI**

This notebook is part of the **“Visual AI in Healthcare with FiftyOne”** series. In this module, we explore how to integrate NVIDIA’s powerful foundation model **VISTA-3D** with **FiftyOne** to visualize, manage, and analyze segmentation results on volumetric CT scan data.

💡 **What you’ll learn in this notebook:**

- How to **load and visualize 3D CT scans** using open-source tools like NiBabel and FiftyOne  
- How to **convert DICOM/NRRD volumes into video samples** for better inspection in FiftyOne  
- How to **enrich CT scan samples with patient and scan metadata**  
- How to **run NVIDIA VISTA-3D** foundation model via API on medical images  
- How to **parse and visualize segmentation masks** returned from the model on each frame  
- How to **explore segmentations interactively** with FiftyOne App

📦 **Dataset**: We use the `dgural/Total-Segmentator-50` dataset hosted on Hugging Face, which includes 50 CT scans and associated metadata.

🔬 **Model**: NVIDIA VISTA-3D foundation model for slice-level segmentation.

📚 **Part of the notebook series:**
1. `01_load_arcade_dataset.ipynb` – Load and visualize the ARCADE dataset.  
2. `02_load_deeplesion_balanced.ipynb` – Curate and balance the DeepLesion dataset.  
3. `03_vlms_analysis_arcade.ipynb` – Use VFMs like NVLabs_CRADIOV3 in dataset undersatnding for ARCADE. 
4. `04_finetune_yolo8_stenosis.ipynb` – Train and integrate YOLOv8 for stenosis detection.  
5. `05_medsam2_ct_scan.ipynb` – Run MedSAM2 on CT scans for segmentation.  
6. `06_nvidia_vista_segmentation.ipynb` – Explore NVIDIA-VISTA-3D.  
7. `07_medgemma_vqa.ipynb` – Perform visual question answering and classification with MedGemma.

All notebooks are standalone but are best experienced sequentially.

> Run this notebook in an environment with ffmpeg, pynrrd, NiBabel, FiftyOne, and API access to NVIDIA VISTA-3D.

### ✅ Requirements

Please install all the requeriments for running this notebook

```
pip install pynrrd nibabel

In [3]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh
import os
import nibabel as nib
import numpy as np
from PIL import Image
import pandas as pd
import requests
import tempfile
import zipfile
import io
import shutil
import nrrd

In [ ]:
# Load TotalSegmentator dataset from Hugging Face
dataset = fouh.load_from_hub("dgural/Total-Segmentator-50")


In [10]:
session = fo.launch_app(dataset, port=5151, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


Download the CT Scans - https://github.com/danielgural/totalsegmentator.git

Follow this notebook: https://voxel51.com/blog/segment-anything-in-a-ct-scan-with-nvidia-vista-3d


In [7]:
def load_ct(scan_name):
    ct_filepath = "TotalSegmentator/" + scan_name + "/ct.nii.gz"
    dir_name = scan_name + "_video"
    new_dir_path = os.path.join(os.path.dirname(ct_filepath), dir_name)
    os.makedirs(new_dir_path, exist_ok=True)

    scan = nib.load(ct_filepath).get_fdata()
    for plane in range(scan.shape[2]):
        p = scan[:,:,plane].astype(np.uint8)
        img = Image.fromarray(p)
        img.save(f'{new_dir_path}/plane{plane}.png')

    mov_in = os.path.join(f'{new_dir_path}/plane%d.png')
    mov_out = os.path.join(f'{new_dir_path}/{scan_name}.mp4')
    !ffmpeg -i {mov_in} -vcodec libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -r 24 -y -an {mov_out}

    sample = fo.Sample(filepath=mov_out)
    sample["ct_filepath"] = os.path.abspath(ct_filepath)
    return sample

In [ ]:
scans = os.listdir("TotalSegmentator")
scans.sort()
df = pd.read_csv("TotalSegmentator/meta.csv", sep=';')

dataset = fo.Dataset(name="TotalSegmentator", overwrite=True)

samples = []
for i, scan in enumerate(scans):
    if i == 51:
        break
    if scan.find(".csv") == -1:
        sample = load_ct(scan)
        row = df[df['image_id'] == scan]
        sample["image_id"] = row["image_id"].item()
        sample["age"] = row["age"].item()
        sample["gender"] = row["gender"].item()
        sample["institute"] = row["institute"].item()
        sample["study_type"] = row["study_type"].item()
        sample["split"] = row["split"].item()
        sample["manufacturer"] = row["manufacturer"].item()
        sample["scanner_model"] = row["scanner_model"].item()
        sample["kvp"] = row["kvp"].item()
        sample["pathology"] = row["pathology"].item()
        sample["pathology_location"] = row["pathology_location"].item()
        samples.append(sample)

dataset.add_samples(samples)
session = fo.launch_app(dataset)

In [ ]:
invoke_url = "https://health.api.nvidia.com/v1/medicalimaging/nvidia/vista-3d"
headers = {
    "Authorization": "Bearer {Your NVIDIA API KEY}",
}

for sample in dataset:
    payload = {
        "prompts": {"classes": None, "points": None},
        "image": f"https://github.com/danielgural/totalsegmentator/raw/main/{sample.image_id}.ct.nii.gz"
    }

    session_req = requests.Session()
    response = session_req.post(invoke_url, headers=headers, json=payload)
    response.raise_for_status()

    try:
        with tempfile.TemporaryDirectory() as temp_dir:
            z = zipfile.ZipFile(io.BytesIO(response.content))
            z.extractall(temp_dir)
            output_path = f"TotalSegmentator/{sample.image_id}/{sample.image_id}.nrrd"
            shutil.move(os.path.join(temp_dir, os.listdir(temp_dir)[0]), output_path)
    except Exception as e:
        with open(f"TotalSegmentator/{sample.image_id}/{sample.image_id}.nrrd", 'w') as file:
            file.write(response.text)

    data, header = nrrd.read(f'TotalSegmentator/{sample.image_id}/{sample.image_id}.nrrd')
    for frame_no, frame in sample.frames.items():
        mask = data[:,:,frame_no-1]
        frame["seg_mask"] = fo.Segmentation(mask=mask)
    sample.save()